In [1]:
import folium
import geopandas as gpd
import numpy as np
import shapely.wkt
from haversine import haversine
from hda import Client
from shapely.geometry import LineString, MultiPolygon, Polygon
from shapely.geometry import Polygon
from shapely.ops import split

In [4]:
#Darmstadt test coordinates, results in result.csv
polygon_darmstadt = shapely.wkt.loads('POLYGON((8.575744628906252 49.997146739553386,8.3807373046875 49.774169585277406,8.91632080078125 49.79013068868653,8.575744628906252 49.997146739553386))')

In [5]:
polygon_spain = shapely.wkt.loads(
    'POLYGON((4.616535050231732 35.87108657413534,-9.749833610246515 35.87108657413534, -9.749833610246515 43.88612553164362, 4.616535050231732 43.88612553164362, 4.616535050231732 35.87108657413534))')

In [2]:
def generateRaster(polygon : shapely.geometry.Polygon, resolution:float) -> gpd.GeoDataFrame:
    """
    Splits a rectangular polygon in smaller rectangular Polygons
    :param polygon: rectangular polygon
    :param resolution: size of smaller polygons in km
    :return: GeoDataframe containing small polygons in the "rectangle" column
    """
    d = {'geometry': [polygon]}
    gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")

    xmin, ymin, xmax, ymax = gdf.total_bounds

    coordinates = shapely.geometry.mapping(polygon)["coordinates"][0]
    x = haversine(coordinates[0], coordinates[1])
    y = haversine(coordinates[0], coordinates[3])
    nx, ny = int(x / resolution), int(y / resolution)

    length = (xmax - xmin) / nx
    wide = (ymax - ymin) / ny

    cols = list(np.arange(xmin, xmax + wide, wide))
    rows = list(np.arange(ymin, ymax + length, length))

    polygons = []
    for x in cols[:-1]:
        for y in rows[:-1]:
            polygons.append(Polygon([(x, y), (x + wide, y), (x + wide, y + length), (x, y + length)]))

    return gpd.GeoDataFrame({'rectangle': polygons})

In [3]:
def polygon_to_bbox(polygon):
    d = {'geometry': [polygon]}
    gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")
    xmin, ymin, xmax, ymax = gdf.total_bounds
    return [xmin, ymin, xmax, ymax]